# Evaluate crypto code localisation techniques

## Setup

Initial setup, such as importing required libraries, starting the database container, etc.

In [ ]:
# Automatically reload modules.
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

# Add project root to Python search path.
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import functions for this notebook.
import notebooks.evaluate_localisation as evaluate_localisation

# Load workspaces from database.
import core.core
import core.workspace

# Widgets.
import ipywidgets as widgets

core.workspace.Workspace.load_from_db(core.core.Core())

# LLDB module
from modules.lldb import LLDBDebuggerModule
lldb_module = LLDBDebuggerModule()
lldb_module.run()

## Configuration

In [ ]:
@widgets.interact(
    drop_database = widgets.Checkbox(value = False, description = 'Drop database'),
    drop_workspace = widgets.Checkbox(value = False, description = 'Drop workspace'),
    rerun_collect_traces = widgets.Checkbox(value = False, description = 'Rerun trace collection'),
    rerun_collect_data = widgets.Checkbox(value = False, description = 'Rerun data collection steps')
)

def update_configuration(**kwargs):
    global CFG
    CFG = kwargs

## Technique

**Attacker goal:** find the encryption code and encryption key.

**Attacker strategy:**
1. Find the basic block(s) implementing the encryption. The following features/techniques are used:
    - *Caballero heuristic*: high amount of bit operations (bitshifts, bitwise XORs, ...).
    - *Linear number of executions*: the number of executions of the basic block must scale linearly with the input size.
    - *High randomness of produced/consumed data*: the data produced or consumed by instructions in the basic block must have high randomness
    - *Differential analysis*: the basic block must be executed when encryption is enabled, and not executed when encryption is disabled (or another encryption algorithm is used).
    
2. Expand the set of encryption basic block(s) from Step 1 by adding basic blocks that fulfill the following requirements:
    - *Close to encryption basic block(s) from Step 1*: The added basic block must have at least one instruction that is "close" to an instruction in an encryption basic block from Step 1. Closeness is calculated using the data dependence graph.

3. Find the instructions that read the encryption key, starting from the encryption-related basic blocks from Step 2. These instructions must adhere to the following:
    - *Part of encryption-related basic block*: the instruction must be a part of one of the encryption-related basic blocks from Step 2.
    - *Instruction values*: The key is much smaller compared to the input, which means that instructions that read the key only read a limited number of different values.
    - *Instruction category*: We filter out some instruction categories that won't use the key (e.g. control flow instructions), and focus on other instruction categories that are more likely to use the key because they are mathematical operations (e.g. bitwise operations, binary operators, etc.)
    
4. Place a breakpoint on the instruction(s) identified in step 3, and print their operands to reconstruct the key.

## Application 1: 7-Zip

7-Zip is a file archiver application that optionally allows encrypting the resulting archives using AES.

The attacker's goal is to find the code implementing AES, and to find the key used.

In [ ]:
app_szip = evaluate_localisation.applications.SzipApplication()
app_szip.create_workspace(clear_db = CFG['drop_database'], clear_workspace = CFG['drop_workspace'])

tec_szip = evaluate_localisation.technique.CryptoLocalisationTechnique(app_szip)

### Collecting traces

In [ ]:
%%time
szip_traces = tec_szip.collect_traces(force_record=CFG['rerun_collect_traces'])

### Step 1: Locating the basic block(s) implementing the encryption

In [ ]:
%%time
if CFG['rerun_collect_data']:
    tec_szip.step_1_localise_crypto_bbls_collect_data(szip_traces)

In [ ]:
%%time
interesting_bbls_szip, basic_blocks_szip = tec_szip.step_1_localise_crypto_bbls_analyse_data(groundtruth_basic_blocks = [
    ('7za', 0x1bc5e, 0x1bd44),
    ('7za', 0x1bd44, 0x1be35),
])

### Step 2: Expanding the basic blocks

In [ ]:
%%time
if CFG['rerun_collect_data']:
    tec_szip.step_2_expand_basic_block_set_collect_data(szip_traces)

In [ ]:
%%time
basic_blocks_to_inspect_szip = tec_szip.step_2_expand_basic_block_set_analyse_data(starting_set = interesting_bbls_szip, max_depth = 1)
print(f'basic_blocks_to_inspect = {basic_blocks_to_inspect_szip}')

### Step 3: Finding the instruction that reads the encryption key

In [ ]:
%%time
if CFG['rerun_collect_data']:
    tec_szip.step_3_find_key_instruction_collect_data(szip_traces, basic_blocks_to_inspect = basic_blocks_to_inspect_szip)

In [ ]:
%%time
instructions_szip = tec_szip.step_3_find_key_instruction_analyse_data(basic_blocks_to_inspect = basic_blocks_to_inspect_szip, groundtruth_instructions = [
    # NOTE: The 'UInt32* w' argument of Aes_Encode at Aes.c:183 contains the subkeys.
    # Also see: https://en.wikipedia.org/wiki/Advanced_Encryption_Standard#High-level_description_of_the_algorithm.
    # 7-Zip uses AES-256, so AES with 14 rounds. Each round uses a different 16-byte roundkey.
    
    # (1) Initial round key addition
    
    # Aes.c lines 189-192: s[i] = src[i] ^ w[i]; for i = 0, 1, 2, 3
    # -> executed one time, with one 16-byte roundkey => operand priority = -1.
    ('7za', 0x1bc2d),
    ('7za', 0x1bc34),
    ('7za', 0x1bc3b),
    ('7za', 0x1bc42),
    
    # (2) 13 Rounds of round key addition, split in 7 + 6 in the implementation of 7-Zip.
    
    # Aes.c line 196: HT16(m, s, 0); --> 4 XORs with w[0 + i]; for i = 0, 1, 2, 3
    # -> executed 7 times, with seven 16-byte roundkeys => operand priority = -7.
    ('7za', 0x1bc65),
    ('7za', 0x1bc9a),
    ('7za', 0x1bccf),
    ('7za', 0x1bd0d),
    
    # Aes.c line 199: HT16(m, s, 4); --> 4 XORs with w[4 + i]; for i = 0, 1, 2, 3
    # -> executed 6 times, with six 16-byte roundkeys => operand priority = -6.
    ('7za', 0x1bd4c),
    ('7za', 0x1bd8a),
    ('7za', 0x1bdbf),
    ('7za', 0x1bdfd),
     
    # (3) Final round key addition
    
    # Aes.c line 203: FT4(0); FT4(1); FT4(2); FT4(3); --> 4 XORs with w[i]; for i = 0, 1, 2, 3
    # -> executed one time, with one 16-byte roundkey => operand priority = -1.
    ('7za', 0x1be84),
    ('7za', 0x1bed0),
    ('7za', 0x1bf20),
    ('7za', 0x1bf60),
], starting_offset = 0x1bc25)

### Step 4: Printing the key using a debugger

In [ ]:
%%time
tec_szip.step_4_print_key_analyse_data(lldb_module, [
    # Instructions of minimum rank:
    ('7za', 0x1bc2d, 'x/4bx "$rdi + 0x10"'),
    ('7za', 0x1bc34, 'x/4bx "$rdi + 0x14"'),
    ('7za', 0x1bc3b, 'x/4bx "$rdi + 0x18"'),
    ('7za', 0x1bc42, 'x/4bx "$rdi + 0x1c"'),
    
    ('7za', 0x1be84, 'x/4bx "$r10 + 0x10"'),
    ('7za', 0x1bed0, 'x/4bx "$r10 + 0x14"'),
    ('7za', 0x1bf20, 'x/4bx "$r10 + 0x18"'),
    ('7za', 0x1bf60, 'x/4bx "$r10 + 0x1c"'),
    
    ('7za', 0x1bd4c, 'x/4bx "$r10 + 0x10"'),
    ('7za', 0x1bd8a, 'x/4bx "$r10 + 0x14"'),
    ('7za', 0x1bdbf, 'x/4bx "$r10 + 0x18"'),
    ('7za', 0x1bdfd, 'x/4bx "$r10 + 0x1c"'),
    
    ('7za', 0x1bc65, 'x/4bx "$r10 + 0x0"'),
    ('7za', 0x1bc9a, 'x/4bx "$r10 + 0x4"'),
    ('7za', 0x1bccf, 'x/4bx "$r10 + 0x8"'),
    ('7za', 0x1bd0d, 'x/4bx "$r10 + 0xc"'),
], max_breakpoint_hits = 128)

We can hence obtain the 14+1 roundkeys used by 7-Zip by looking at the values read by these instructions:

```
roundkey  1/15: 0x65 0x11 0xf1 0x50 0x7a 0x3c 0x2a 0x98 0x2a 0xe6 0xde 0xf4 0xf6 0x58 0x18 0xed
roundkey  2/15: 0x6c 0xf2 0x42 0x65 0x1a 0xca 0x19 0xd1 0x5c 0xeb 0xc6 0xa6 0x7a 0xe2 0x89 0x1d
roundkey  3/15: 0xfc 0xb6 0x55 0x8a 0x86 0x8a 0x7f 0x12 0xac 0x6c 0xa1 0xe6 0x5a 0x34 0xb9 0x0b
roundkey  4/15: 0xd2 0xea 0x14 0x4e 0xc8 0x20 0x0d 0x9f 0x94 0xcb 0xcb 0x39 0xee 0x29 0x42 0x24
roundkey  5/15: 0x5b 0x9a 0x63 0xa2 0xdd 0x10 0x1c 0xb0 0x71 0x7c 0xbd 0x56 0x2b 0x48 0x04 0x5d
roundkey  6/15: 0x23 0xb8 0xe6 0x02 0xeb 0x98 0xeb 0x9d 0x7f 0x53 0x20 0xa4 0x91 0x7a 0x62 0x80
roundkey  7/15: 0x85 0x30 0xae 0x23 0x58 0x20 0xb2 0x93 0x29 0x5c 0x0f 0xc5 0x02 0x14 0x0b 0x98
roundkey  8/15: 0x54 0x42 0xcd 0x44 0xbf 0xda 0x26 0xd9 0xc0 0x89 0x06 0x7d 0x51 0xf3 0x64 0xfd
roundkey  9/15: 0x80 0x73 0xfa 0xf2 0xd8 0x53 0x48 0x61 0xf1 0x0f 0x47 0xa4 0xf3 0x1b 0x4c 0x3c
roundkey 10/15: 0x59 0xed 0xe4 0xaf 0xe6 0x37 0xc2 0x76 0x26 0xbe 0xc4 0x0b 0x77 0x4d 0xa0 0xf6
roundkey 11/15: 0x73 0x93 0xb8 0x07 0xab 0xc0 0xf0 0x66 0x5a 0xcf 0xb7 0xc2 0xa9 0xd4 0xfb 0xfe
roundkey 12/15: 0x8a 0xa5 0xeb 0x14 0x6c 0x92 0x29 0x62 0x4a 0x2c 0xed 0x69 0x3d 0x61 0x4d 0x9f
roundkey 13/15: 0xbc 0x70 0x63 0x20 0x17 0xb0 0x93 0x46 0x4d 0x7f 0x24 0x84 0xe4 0xab 0xdf 0x7a
roundkey 14/15: 0xe3 0xc7 0x75 0xce 0x8f 0x55 0x5c 0xac 0xc5 0x79 0xb1 0xc5 0xf8 0x18 0xfc 0x5a
roundkey 15/15: 0x51 0xc0 0xdd 0x61 0x46 0x70 0x4e 0x27 0x0b 0x0f 0x6a 0xa3 0xef 0xa4 0xb5 0xd9
```

You can also verify that these are AES roundkeys, e.g. using [aeskeyschedule](https://pypi.org/project/aeskeyschedule/).
Since 7-Zip uses AES-256, the AES key is 256 bits (= 2x 16-byte roundkeys) long, so the key is:

```
roundkey  1/15: 0x65 0x11 0xf1 0x50 0x7a 0x3c 0x2a 0x98 0x2a 0xe6 0xde 0xf4 0xf6 0x58 0x18 0xed
roundkey  2/15: 0x6c 0xf2 0x42 0x65 0x1a 0xca 0x19 0xd1 0x5c 0xeb 0xc6 0xa6 0x7a 0xe2 0x89 0x1d
```

We can use `aeskeyschedule` to print the other round keys:

```bash
$ pip3 install aeskeyschedule --user --upgrade
$ aeskeyschedule '6511f1507a3c2a982ae6def4f65818ed6cf242651aca19d15cebc6a67ae2891d'
 0: 6511f1507a3c2a982ae6def4f65818ed
 1: 6cf242651aca19d15cebc6a67ae2891d
 2: fcb6558a868a7f12ac6ca1e65a34b90b
 3: d2ea144ec8200d9f94cbcb39ee294224
 4: 5b9a63a2dd101cb0717cbd562b48045d
 5: 23b8e602eb98eb9d7f5320a4917a6280
 6: 8530ae235820b293295c0fc502140b98
 7: 5442cd44bfda26d9c089067d51f364fd
 8: 8073faf2d8534861f10f47a4f31b4c3c
 9: 59ede4afe637c27626bec40b774da0f6
10: 7393b807abc0f0665acfb7c2a9d4fbfe
11: 8aa5eb146c9229624a2ced693d614d9f
12: bc70632017b093464d7f2484e4abdf7a
13: e3c775ce8f555cacc579b1c5f818fc5a
14: 51c0dd6146704e270b0f6aa3efa4b5d9
```

To verify whether we have found the right key, we add the following excerpt to the end of `CKeyInfo::CalcKey()` in `CPP/7zip/Crypto/7zAes.cpp`, which implements the KDF:

```cpp
  std::cout << "KEY: ";
  for (unsigned int i = 0; i < kKeySize; ++i) {
      std::cout << std::hex << (int)Key[i] << ' ';
  }
  std::cout << std::endl;
```

which prints the following to stdout:

```
KEY: 65 11 f1 50 7a 3c 2a 98 2a e6 de f4 f6 58 18 ed 6c f2 42 65 1a ca 19 d1 5c eb c6 a6 7a e2 89 1d 
```

## Application 2: GPG

GNU Privacy Guard (GnuPG or GPG for short) is a free application that implements the OpenPGP standard for encrypting and decrypting data.

It supports several asymmetric cryptography algorithms (e.g. RSA, ElGamal, DSA,  ECDH, ECDSA), symmetric ciphers (e.g. 3DES, Blowfish, Twofish, AES), and cryptographic hash algorithms (e.g. SHA1, SHA2). GPG also supports optional compression (e.g. ZIP, ZLIB, BZIP2).

We will encrypt a file using the symmetric AES cipher. The attacker's goal is then to find the code implementing AES, and to find the key that is used.

In [ ]:
app_gpg = evaluate_localisation.applications.GpgApplication()
app_gpg.create_workspace(clear_db = CFG['drop_database'], clear_workspace = CFG['drop_workspace'])

tec_gpg = evaluate_localisation.technique.CryptoLocalisationTechnique(app_gpg)

### Collecting traces

In [ ]:
%%time
gpg_traces = tec_gpg.collect_traces(force_record=CFG['rerun_collect_traces'])

### Step 1: Locating the basic block(s) implementing the encryption

In [ ]:
%%time
if CFG['rerun_collect_data']:
    tec_gpg.step_1_localise_crypto_bbls_collect_data(gpg_traces)

In [ ]:
%%time
interesting_bbls_gpg, basic_blocks_gpg = tec_gpg.step_1_localise_crypto_bbls_analyse_data(groundtruth_basic_blocks = [
    ('libgcrypt.so.20', 0x411b0, 0x41200),
    ('libgcrypt.so.20', 0x41200, 0x412cb),
    ('libgcrypt.so.20', 0x412cb, 0x412f8),
])

### Step 2: Expanding the basic blocks

In [ ]:
%%time
if CFG['rerun_collect_data']:
    tec_gpg.step_2_expand_basic_block_set_collect_data(gpg_traces)

In [ ]:
%%time
basic_blocks_to_inspect_gpg = tec_gpg.step_2_expand_basic_block_set_analyse_data(starting_set = interesting_bbls_gpg, max_depth = 2)
print(f'basic_blocks_to_inspect = {basic_blocks_to_inspect_gpg}')

### Step 3: Finding the instruction that reads the encryption key

In [ ]:
%%time
if CFG['rerun_collect_data']:
    tec_gpg.step_3_find_key_instruction_collect_data(gpg_traces, basic_blocks_to_inspect = basic_blocks_to_inspect_gpg)

In [ ]:
%%time
instructions_gpg = tec_gpg.step_3_find_key_instruction_analyse_data(basic_blocks_to_inspect = basic_blocks_to_inspect_gpg, groundtruth_instructions = [
    ('libgcrypt.so.20', 0x411f2), # pxor xmm2, xmmword [rdx]
    ('libgcrypt.so.20', 0x4120a), # pxor xmm4, xmmword [rdx]
    ('libgcrypt.so.20', 0x412d8), # pxor xmm4, xmmword [rdx]
], starting_offset = 0x426e0)

### Step 4: Printing the key using a debugger

In [ ]:
%%time
tec_gpg.step_4_print_key_analyse_data(lldb_module, [
    ('libgcrypt.so.20', 0x4120a, 'x/16bx "$rdx"'),
    ('libgcrypt.so.20', 0x411f2, 'x/16bx "$rdx"'),
    ('libgcrypt.so.20', 0x412d8, 'x/16bx "$rdx"'),
], max_breakpoint_hits = 64)

Like for 7-Zip, you can reconstruct the round keys from these values.

**However, by default, GPG uses a salted S2K function to derive an encryption key from the passphrase (`--s2k-mode 3`). This results in the key being different for different runs.**

For more details, see Section 5.3 of RFC4880:
>   If the encrypted session key is not present (which can be detected
>   on the basis of packet length and S2K specifier size), then the S2K
>   algorithm applied to the passphrase produces the session key for
>   decrypting the file, using the symmetric cipher algorithm from the
>   Symmetric-Key Encrypted Session Key packet.
>
>   [...]
>
>   Note: because an all-zero IV is used for this decryption, the S2K
>   specifier MUST use a salt value, either a Salted S2K or an
>   Iterated-Salted S2K.  The salt value will ensure that the decryption
>   key is not repeated even if the passphrase is reused.

To verify this is the key, you can also add the following function to `libgcrypt` at the top of `cipher/rijndael-ssse3-amd64.c` to print the key, but beware that the key is different for each run:
```c
static void aes_print_key(const RIJNDAEL_context *ctx)
{
  // Save all XMM registers so we don't overwrite them.
  unsigned char regsavearea[16*16];

  asm volatile(
          "movdqu %%xmm0, 0*16(%0)\n"
          "movdqu %%xmm1, 1*16(%0)\n"
          "movdqu %%xmm2, 2*16(%0)\n"
          "movdqu %%xmm3, 3*16(%0)\n"
          "movdqu %%xmm4, 4*16(%0)\n"
          "movdqu %%xmm5, 5*16(%0)\n"
          "movdqu %%xmm6, 6*16(%0)\n"
          "movdqu %%xmm7, 7*16(%0)\n"
          "movdqu %%xmm8, 8*16(%0)\n"
          "movdqu %%xmm9, 9*16(%0)\n"
          "movdqu %%xmm10, 10*16(%0)\n"
          "movdqu %%xmm11, 11*16(%0)\n"
          "movdqu %%xmm12, 12*16(%0)\n"
          "movdqu %%xmm13, 13*16(%0)\n"
          "movdqu %%xmm14, 14*16(%0)\n"
          "movdqu %%xmm15, 15*16(%0)\n"
          :
          : "r" (regsavearea)
          : "memory");

  // Print the key to stderr.
  for (int round = 0; round < ctx->rounds; ++round)
  {
    fprintf(stderr, "Round %d/%d key: ", round, ctx->rounds);

    for (int i = 0; i < 4; ++i)
    {
      for (int j = 0; j < 4; ++j)
      {
        fprintf(stderr, "%02x ", ctx->keyschenc[round][i][j]);
      }
    }

    fprintf(stderr, "\n");
  }

  asm volatile(
          "movdqu 0*16(%0), %%xmm0\n"
          "movdqu 1*16(%0), %%xmm1\n"
          "movdqu 2*16(%0), %%xmm2\n"
          "movdqu 3*16(%0), %%xmm3\n"
          "movdqu 4*16(%0), %%xmm4\n"
          "movdqu 5*16(%0), %%xmm5\n"
          "movdqu 6*16(%0), %%xmm6\n"
          "movdqu 7*16(%0), %%xmm7\n"
          "movdqu 8*16(%0), %%xmm8\n"
          "movdqu 9*16(%0), %%xmm9\n"
          "movdqu 10*16(%0), %%xmm10\n"
          "movdqu 11*16(%0), %%xmm11\n"
          "movdqu 12*16(%0), %%xmm12\n"
          "movdqu 13*16(%0), %%xmm13\n"
          "movdqu 14*16(%0), %%xmm14\n"
          "movdqu 15*16(%0), %%xmm15\n"
          :
          : "r" (regsavearea)
          : "memory");
}
```
and call it at the start of `do_vpaes_ssse3_enc`.

Note that, unlike 7-Zip, you cannot use `aeskeyschedule` for verification, as the SSSE3 algorithm that is used for AES in libgcrypt uses a custom key schedule.
For more information, see [1].

There is also a proof-of-concept implementation available from the author at https://crypto.stanford.edu/vpaes/.
You can add the following in `test/test.c` to verify the key:

```
  aes_key
    *key  = aes_schedule((byte *) ikey, key_size, NULL,
			 AES_DIRECTION_ENCRYPT),

    *dkey = aes_schedule((byte *) ikey, key_size, NULL,
			 AES_DIRECTION_DECRYPT);

    for (int i = 0; i < 15; ++i) {
        unsigned char buf[16];
        __m128i subkey = key->_subkey[i];
        memcpy(buf, &subkey, sizeof(buf));

        printf("key %i/15: ", i);

        for (int j = 0; j < 16; ++j) {
            printf("%x ", buf[j]);
        }

        printf("\n");
    }
```
and then pass the original key we get with libgcrypt in AESNI mode (and without salting in S2K!):
```bash
./test -K e5e9fa1ba31ecd1ae84f75caaa474f3a663f05f4848b7772d9ac64936354c4ea
```

which outputs the following values:
```
key 0/15: 68 f8 ef 1f a7 f7 1c 6f 88 b7 a4 de df bf b7 13
key 1/15: 1d c6 69 1f 60 55 a 66 39 aa 46 cc 35 9d f7 fb
key 2/15: 52 35 9a 61 d6 4e a5 b6 1b 13 44 b6 0 c7 37 fa
key 3/15: 15 6a dd 25 2e ac 8c cb 4c 31 43 8f 22 9b ef b2
key 4/15: 48 72 88 2 c5 ee 76 a3 85 80 69 99 de 95 5 74
key 5/15: f2 a5 b6 b7 87 9 61 5e 90 38 84 20 e9 cf 30 63
key 6/15: d3 c5 37 28 4d b 1a 7 93 ab 28 12 16 1e 76 ea
key 7/15: 1 d fc 5b dd 99 c6 c9 16 67 7a 25 a4 6e 11 20
key 8/15: 0 50 cd 52 16 15 8c e3 de 8b ff 90 93 db d2 cc
key 9/15: 73 f5 5d 9b f5 5d c0 e0 b8 b b0 e0 47 c9 fa e5
key 10/15: c5 74 97 2b 88 f4 40 bc d ea e4 58 c5 a4 6d e0
key 11/15: fc aa c0 99 52 38 5b 59 b1 96 67 99 ad fa c6 22
key 12/15: c6 a8 df 2c 15 57 c4 97 43 78 7b e7 dd d7 4d 0
key 13/15: d5 f2 4f 69 dc ad 4f 10 36 5c 49 d2 c0 3f d4 ab
key 14/15: f8 e2 c1 48 b2 8d b6 85 f5 86 a9 66 24 2b 74 7a
```
which match the values read by the groundtruth instructions.

[1] M. Hamburg, “Accelerating AES with Vector Permute Instructions,” in Cryptographic Hardware and Embedded Systems - CHES 2009, vol. 5747, C. Clavier and K. Gaj, Eds. Berlin, Heidelberg: Springer Berlin Heidelberg, 2009, pp. 18–32. doi: 10.1007/978-3-642-04138-9_2.

## Ablation Study

### 7-Zip

In [ ]:
app_szip = evaluate_localisation.applications.SzipApplication()
app_szip.create_workspace(clear_db = CFG['drop_database'], clear_workspace = CFG['drop_workspace'])

tec_szip = evaluate_localisation.technique.CryptoLocalisationTechnique(app_szip)

In [ ]:
%%time
szip_traces = tec_szip.collect_traces(force_record=CFG['rerun_collect_traces'])

In [ ]:
%%time
if CFG['rerun_collect_data']:
    tec_szip.step_1_localise_crypto_bbls_collect_data(szip_traces)

In [ ]:
%%time
if CFG['rerun_collect_data']:
    tec_szip.step_2_expand_basic_block_set_collect_data(szip_traces)

In [ ]:
%%time
szip_step_1_ablation_results = tec_szip.run_ablation_step_1(expected_basic_blocks=[
    ('7za', 0x1bc5e, 0x1bd44),
    ('7za', 0x1bd44, 0x1be35),
]);

In [ ]:
%%time
szip_step_2a_ablation_data,szip_step_2a_ablation_results = tec_szip.run_ablation_step_2a(step_1_ablation_results=szip_step_1_ablation_results, expected_basic_blocks=[
    ('7za', 0x1bc5e, 0x1bd44),
    ('7za', 0x1bd44, 0x1be35),
], data_deps_depth=1);

In [ ]:
ablation_szip_bbls_to_inspect = set()
for k,v in szip_step_2a_ablation_results.items():
    ablation_szip_bbls_to_inspect=ablation_szip_bbls_to_inspect.union(v)

In [ ]:
%%time
if CFG['rerun_collect_data']:
    tec_szip.step_3_find_key_instruction_collect_data(szip_traces, basic_blocks_to_inspect = ablation_szip_bbls_to_inspect)

In [ ]:
%%time
szip_step_2b_ablation_results = tec_szip.run_ablation_step_2b(step_2a_data = szip_step_2a_ablation_data,step_2a_results = szip_step_2a_ablation_results,expected_instructions=[
    ('7za', 0x1bc2d),
    ('7za', 0x1bc34),
    ('7za', 0x1bc3b),
    ('7za', 0x1bc42),
    ('7za', 0x1bc65),
    ('7za', 0x1bc9a),
    ('7za', 0x1bccf),
    ('7za', 0x1bd0d),
    ('7za', 0x1bd4c),
    ('7za', 0x1bd8a),
    ('7za', 0x1bdbf),
    ('7za', 0x1bdfd),
    ('7za', 0x1be84),
    ('7za', 0x1bed0),
    ('7za', 0x1bf20),
    ('7za', 0x1bf60),
])

In [ ]:
%%time
tec_szip.run_ablation_step_3(step_2_ablation_results=szip_step_2_ablation_results, expected_instructions=[
    ('7za', 0x1bc2d),
    ('7za', 0x1bc34),
    ('7za', 0x1bc3b),
    ('7za', 0x1bc42),
    ('7za', 0x1bc65),
    ('7za', 0x1bc9a),
    ('7za', 0x1bccf),
    ('7za', 0x1bd0d),
    ('7za', 0x1bd4c),
    ('7za', 0x1bd8a),
    ('7za', 0x1bdbf),
    ('7za', 0x1bdfd),
    ('7za', 0x1be84),
    ('7za', 0x1bed0),
    ('7za', 0x1bf20),
    ('7za', 0x1bf60),
])

### GPG

In [ ]:
app_gpg = evaluate_localisation.applications.GpgApplication()
app_gpg.create_workspace(clear_db = CFG['drop_database'], clear_workspace = CFG['drop_workspace'])

tec_gpg = evaluate_localisation.technique.CryptoLocalisationTechnique(app_gpg)

In [ ]:
%%time
gpg_traces = tec_gpg.collect_traces(force_record=CFG['rerun_collect_traces'])

In [ ]:
%%time
if CFG['rerun_collect_data']:
    tec_gpg.step_1_localise_crypto_bbls_collect_data(gpg_traces)

In [ ]:
%%time
if CFG['rerun_collect_data']:
    tec_gpg.step_2_expand_basic_block_set_collect_data(gpg_traces)

In [ ]:
%%time
gpg_step_1_ablation_results = tec_gpg.run_ablation_step_1(expected_basic_blocks=[
    ('libgcrypt.so.20', 0x411b0, 0x41200),
    ('libgcrypt.so.20', 0x41200, 0x412cb),
    ('libgcrypt.so.20', 0x412cb, 0x412f8),
]);

In [ ]:
%%time
gpg_step_2a_ablation_data,gpg_step_2a_ablation_results = tec_gpg.run_ablation_step_2a(step_1_ablation_results=gpg_step_1_ablation_results, expected_basic_blocks=[
    ('libgcrypt.so.20', 0x411b0, 0x41200),
    ('libgcrypt.so.20', 0x41200, 0x412cb),
    ('libgcrypt.so.20', 0x412cb, 0x412f8),
], data_deps_depth=2);

In [ ]:
ablation_gpg_bbls_to_inspect = set()
for k,v in gpg_step_2a_ablation_results.items():
    ablation_gpg_bbls_to_inspect=ablation_gpg_bbls_to_inspect.union(v)

In [ ]:
%%time
if CFG['rerun_collect_data']:
    tec_gpg.step_3_find_key_instruction_collect_data(gpg_traces, basic_blocks_to_inspect = ablation_gpg_bbls_to_inspect)

In [ ]:
%%time
gpg_step_2b_ablation_results = tec_gpg.run_ablation_step_2b(step_2a_data = gpg_step_2a_ablation_data,step_2a_results = gpg_step_2a_ablation_results,expected_instructions=[
    ('libgcrypt.so.20', 0x411f2),
    ('libgcrypt.so.20', 0x4120a),
    ('libgcrypt.so.20', 0x412d8),
]);

In [ ]:
%%time
tec_gpg.run_ablation_step_3(step_2_ablation_results=gpg_step_2a_ablation_results, expected_instructions=[
    ('libgcrypt.so.20', 0x411f2),
    ('libgcrypt.so.20', 0x4120a),
    ('libgcrypt.so.20', 0x412d8),
])